In [ ]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.
df.head()

df.describe()

df.info()

df.isna().sum()

In [ ]:
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [ ]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
X_train_full, X_test_full, y_train, y_test = train_test_split(df, y, test_size = 0.2, random_state = 42)

In [ ]:
#TASK 5 - 1. Create a pipeline using Gaussian Naive Bayes
#         2. Fit the model to the training data
#         3. Predict values for test set
#         4. Print accuracy score
#from sklearn.naive_bayes import GaussianNB
#start_time = time.time()
#pipeline = Pipeline([('scaler', StandardScaler()), ('classifier', GaussianNB())])
#pipeline.fit(X_train_full, y_train)
#y_pred = pipeline.predict(X_test_full)
#accuracy = accuracy_score(y_test, y_pred)
#end_time = time.time()
#total_time = end_time - start_time
#print(f"Accuracy: {accuracy}")

In [ ]:
#TASK 6 - 1. Note the start time before defining the pipeline
#         2. Note the end time and report the difference as the time taken by the model training and inference.
#total_time = end_time - start_time
#print(f"Total time taken for model training and inference: {total_time:.4f} seconds")

In [ ]:
# SOLUTION FOR TASK 5 and 6

# Define a Pipeline with Gaussian Naive Bayes and train it on the full dataset
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train_full, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test_full)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)

In [ ]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(df.T)  # Transpose to treat features as data points
selected_features_indices = []
for cluster_label in range(n_clusters):
    feature_indices_in_cluster = np.where(kmeans.labels_ == cluster_label)[0]
    selected_features_indices.append(feature_indices_in_cluster[0])
selected_features = df.columns[selected_features_indices]

In [ ]:
#TASK 8 - Train another model (GaussianNB) on the new dataset, and report time taken and accuracy

start_time = time.time()
classifier_pipeline_selected = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_selected.fit(X_train_full[selected_features], y_train)
y_pred_selected = classifier_pipeline_selected.predict(X_test_full[selected_features])
end_time = time.time()
selected_features_time = end_time - start_time
accuracy_selected = accuracy_score(y_test, y_pred_selected)
print(f"Accuracy with selected features: {accuracy_selected}")
print(f"Total time taken with selected features: {selected_features_time:.4f} seconds")